# Research Question
## What is the effect of age on continued physical activity during COVID-19 lockdowns?
### Author: Jan Rombouts

This notebook provides and analysis of data generated by the Dutch (CBS) and Canadian national (StatCan) surveys.
Code in this notebook depends on Pandas, NumPy, and Bokeh.

In [370]:
import pandas as pd
import numpy as np

## Import files and Clean data

### Canada

This CCHS survey had ~65k respondents in 2018 and 2020.

In [371]:
canada = (pd.read_csv("StatCan_CA_activity_levels_2018-2020.csv") # Import Canada csv file
            .loc[18:]
            .drop(columns=["DGUID", "UOM", "UOM_ID", "SCALAR_FACTOR", "SCALAR_ID", "VECTOR",
                           "COORDINATE", "STATUS", "SYMBOL", "TERMINATED", "DECIMALS",
                           "Indicators", "Sex"]) # Drop columns
            .rename(columns={"REF_DATE":"year",
                             "GEO":"country",
                             "Age group":"age group",
                             "VALUE":">2.5 hours physical activities per week (%)"})) # Rename columns

canada = canada.loc[(canada["year"] == 2018) | (canada["year"] == 2020)] # Remove 2019 because there is no data of it

# Pivot the table to get 95% CI and average in separate columns
canada = pd.pivot_table(canada, values = '>2.5 hours physical activities per week (%)',
                index=['age group', 'year'], columns=["Characteristics"]).reset_index()

# Calculate standard error with the 95% CI
n_canada = 65000
canada["standard error"] = ((canada["High 95% confidence interval, percent"] - canada["Low 95% confidence interval, percent"])/ 3.92)

# Drop 95% CI columns and rename percent to activity level
canada = canada.drop(columns=["High 95% confidence interval, percent", "Low 95% confidence interval, percent"])
canada = canada.rename(columns={"Percent":"activity level"})
canada["country"] = "Canada"

### The Netherlands

This CBS survey had ~10k respondents in 2019 and 2020

In [388]:
holland = (pd.read_excel("CBS_NL_activity_levels_2019-2020.xlsx", header=3) # Import Dutch activity file
           .iloc[1:] # Select relevant rows
           .rename(columns={"Unnamed: 1":"Characteristics", "Unnamed: 2":"year",
                    "Unnamed: 0":"age group"}))

# Filter out age groups that are not used in analyses
holland = holland[(holland["age group"] != "Leeftijd: 0 tot 4 jaar") & (holland["age group"] != "Leeftijd: 0 tot 12 jaar") &
                   (holland["age group"] != "Leeftijd: 12 tot 16 jaar")]

# Rename age groups
age_dict = {"Leeftijd: 0 tot 4 jaar":"0 to 3 years",
           "Leeftijd: 0 tot 12 jaar":"0 to 11 years",
           "Leeftijd: 4 tot 12 jaar":"4 to 11 years",
           "Leeftijd: 12 tot 16 jaar":"12 to 15 years",
           "Leeftijd: 16 tot 20 jaar":"18 to 19 years",
           "Leeftijd: 20 tot 30 jaar":"20 to 29 years",
           "Leeftijd: 30 tot 40 jaar":"30 to 39 years",
           "Leeftijd: 40 tot 50 jaar":"40 to 49 years",
           "Leeftijd: 50 tot 55 jaar":"50 to 54 years",
           "Leeftijd: 55 tot 65 jaar":"55 to 64 years",
           "Leeftijd: 65 tot 75 jaar":"65 to 74 years",
           "Leeftijd: 75 jaar of ouder":"75 years and over",
           "Leeftijd: 12 tot 18 jaar":"12 to 17 years",
           "Leeftijd: 18 jaar of ouder":"18 years and over",
           "Totaal personen":"All ages"}
holland["age group"]= holland["age group"].map(age_dict)

# Change datatype
holland["Bewegen, 4 jaar of ouder |Beweegrichtlijn|Voldoende matig intensieve inspanning"] = pd.to_numeric(
            holland["Bewegen, 4 jaar of ouder |Beweegrichtlijn|Voldoende matig intensieve inspanning"].str.replace(",","."))

# Pivot the table to get 95% CI and average in separate columns
holland = pd.pivot_table(holland, values = 'Bewegen, 4 jaar of ouder |Beweegrichtlijn|Voldoende matig intensieve inspanning',
               index=['age group', 'year'], columns=["Characteristics"]).reset_index()

# Calculate standard error with the 95% CI
n_holland = 10000
holland["standard error"] = ((holland["Bovengrens 95%-interval"] - 
                              holland["Ondergrens 95%-interval"])/ 3.92)

# Drop 95% CI columns and rename percent to activity level
holland = holland.drop(columns=["Bovengrens 95%-interval", "Ondergrens 95%-interval"])
holland = holland.rename(columns={"Waarde":"activity level"}).iloc[:18]

# Merge data of Canada and the Netherlands
To merge these to dataframes, equal age groups are needed. The following age groups were chosen (up to and including):
- Children:             4-11  years old
- Teens:                12-17 years old
- Young adults:         18-34 years old
- Middle aged adults:   35-49 years old
- Older adults:         50-64 years old
- 65+:                  65 years old and over

These follow Canada's age groups. Some of the Netherlands' age groups are the same already but most groups have to be reformed so they have the same cutoffs. This was done by using the population numbers of the Netherlands by age. Then the following processing steps were performed:

- Cleaning of population file and prepare for merging

In [389]:
# Import file
holland_population = (pd.read_excel("CBS_NL_Population_2019_20.xlsx", header=3) # Import Dutch population file
           .iloc[4:101] # Select relevant rows
           .rename(columns={"Perioden":"age"}))

# Extract ages from age column without "jaar"
holland_population[["age", "drop"]] = holland_population["age"].str.split(" ", n=1, expand=True)
holland_population = holland_population.drop("drop", axis=1)
holland_population["age"] = pd.to_numeric(holland_population["age"])

# Change year from column to rows
holland_population = holland_population.melt(id_vars=["age"], value_vars=[2019, 2020], value_name="year_size")
holland_population = holland_population.rename(columns={"variable":"year"})

- Preparing holland df for merging

In [374]:
# Split the age groups into two column > start and end of the age group
holland[["age group start", "age group end"]] = holland["age group"].str.split("to", n=1, expand=True)
holland["age group start"] = holland["age group start"].str.extract(r'(\d+)', expand=True)
holland["age group end"] = holland["age group end"].str.extract(r'(\d+)', expand=True)

# Change dtypes
holland["age group start"] = pd.to_numeric(holland["age group start"])
holland["age group end"] = pd.to_numeric(holland["age group end"]).fillna(100)

- Assigning the activity levels percentages of the _original_ age groups to the respective ages in that group
    -For example: age group 30-39 in 2019 has 53.9% active people, so age 30 will also have 53.9% active people

In [375]:
# Create new df
age_df = pd.DataFrame()
groups = []
activity_levels = []
years = []
standard_errors = []

# Loop to make lists with all ages separately of every age group
for time in holland["year"].unique(): # iterates through all age groups for 2019 and 2020, creates df with data of 1 year
    year_df = holland[holland["year"] == time]
    for group_start in year_df["age group start"]: # iterates through age group starts, finds end of group and activity levels
        
        # Finds group end, creates list with all ages from start to end. Adds these to a list
        group_end = year_df["age group end"][year_df.index[year_df["age group start"] == group_start]].iloc[0]
        group_list = np.arange(group_start, (group_end+1), 1)
        groups.extend(group_list)
        
        # Finds activity level corresponding to age group, creates list with activity level for every age. Adds these to a list
        activity_level = (year_df["activity level"]
                          [year_df.index[year_df["age group start"] == group_start]].iloc[0])
        activity_list = np.repeat(activity_level, len(group_list))
        activity_levels.extend(activity_list)
        
        # Finds standard error corresponding to age group, creates list with activity level for every age. Adds these to a list
        standard_error = (year_df["standard error"]
                          [year_df.index[year_df["age group start"] == group_start]].iloc[0])
        standard_list = np.repeat(standard_error, len(group_list))
        standard_errors.extend(standard_list)
        
        # Finds year corresponding to age group, creates list with year for every age. Adds these to a list
        year_list = np.repeat(time, len(group_list))
        years.extend(year_list)
        
# Finally add the group, activity level, and year lists to a dataframe
age_df["age"] = groups
age_df["activity_levels"] = activity_levels
age_df["standard error"] = standard_errors
age_df["year"] = years

- Multiply the % active people by the total number of people of each age

In [376]:
# Merge on year and age, creating a df with activity levels, age, and size of each age for 2019 and 2020
merged_df = age_df.merge(holland_population, on=["year", "age"], how="left")

# Change dtypes to allow for multiplications
merged_df["year_size"] = pd.to_numeric(merged_df["year_size"])
merged_df["activity_levels"] = pd.to_numeric(merged_df["activity_levels"])
merged_df["active_people"] = round((merged_df["activity_levels"]/100) * merged_df["year_size"])

- Create small sub-dataframes with the new redistributed age groups
- Calculate the new percentage of active people by summing active people and dividing by summed total people of the new group

In [377]:
# Create dictionary with new groups
age_groups_dict = {"12 to 17 years":np.arange(12,18,1),
                   "18 to 34 years":np.arange(18,35,1),
                   "35 to 49 years":np.arange(35,50,1),
                   "50 to 64 years":np.arange(50,65,1),
                   "65 years and over":np.arange(65,101,1)}

# Create new df
new_holland = pd.DataFrame()
groups = []
years = []
percentage = []
standard_error = []

# Loop to create four new lists with year, new age groups, percentages, and standard error
for time in merged_df["year"].unique(): # iterates through all age groups for 2019 and 2020
    sub_group_df = merged_df[merged_df["year"] == time]
    for key in age_groups_dict: # iterates through all age groups in dictionary
        age_group = key
        sub_group = sub_group_df.loc[sub_group_df["age"].isin(age_groups_dict[key])]
        new_group = round((sub_group["active_people"].sum() / sub_group["year_size"].sum() *100),1)
        se_new_group = (sub_group["standard error"].sum() / len(sub_group))
        standard_error.append(se_new_group)
        groups.append(age_group)
        years.append(time)
        percentage.append(new_group)

# Add three lists as column to new dataframe
new_holland["age group"] = groups
new_holland["year"] = years
new_holland["activity level"] = percentage
new_holland["standard error"] = standard_error
new_holland["country"] = "the Netherlands"

<font color='red'> # NOTE: standard error calculation is quite rudimentary and incorrect. I could not find a correct (and not too time consuming) way to do it for this assignment but I am aware it is not good practice #</font>

## Finally, merge Canada and the Netherlands

Because the years _before_ lockdowns are different (2018 and 2019), the year names were changed to "pre" and "lockdown"

In [378]:
year_dict = {2018:"pre", 2019:"pre", 2020:"lockdown"}
new_holland["year"]= new_holland["year"].map(year_dict)
canada["year"] = canada["year"].map(year_dict)

In [379]:
# Then merge the two datafiles
final_df = pd.concat([new_holland, canada])
final_df.sort_values(["year", "age group"])

,age group,year,activity level,standard error,country
5,12 to 17 years,lockdown,42.4,2.040816,the Netherlands
1,12 to 17 years,lockdown,42.9,1.403061,Canada
6,18 to 34 years,lockdown,62.2,1.593637,the Netherlands
3,18 to 34 years,lockdown,59.9,1.071429,Canada
7,35 to 49 years,lockdown,60.5,1.513605,the Netherlands
5,35 to 49 years,lockdown,57.2,1.020408,Canada
8,50 to 64 years,lockdown,59.9,1.607143,the Netherlands
7,50 to 64 years,lockdown,55.1,0.918367,Canada
9,65 years and over,lockdown,56.0,1.632653,the Netherlands
9,65 years and over,lockdown,40.3,0.561224,Canada


# Data inspection

For the preliminary data inspection, we will use the final_df. When we go into answering the research question, a pivotted dataframe will be used, pivot_df, to review changes between pre and lockdown

# Check normality

In [380]:
from scipy import stats

shapiro_test = stats.shapiro(final_df["activity level"])
shapiro_test

ShapiroResult(statistic=0.8502222299575806, pvalue=0.005376183893531561)

Data does not seem to be normally distributed according to the Shapiro-Wilks test

# Check homogeneity of variance

In [381]:
import hvplot.pandas
boxplot_activity = final_df.hvplot.box(y="activity level", by=["age group"])
boxplot_activity

:BoxWhisker   [age group]   (activity level)

In [382]:
# Levene's test
from scipy.stats import levene
levene(final_df['activity level'][final_df['age group'] == '12 to 17 years'],
               final_df['activity level'][final_df['age group'] == '18 to 34 years'],
               final_df['activity level'][final_df['age group'] == '35 to 49 years'],
               final_df['activity level'][final_df['age group'] == '50 to 64 years'],
               final_df['activity level'][final_df['age group'] == '65 years and over'],
       center="median")

LeveneResult(statistic=3.7003593244699986, pvalue=0.027435222853057988)

The boxplots and Levene's test suggest there is equal variance between age groups

# Analysis of Variance

## To answer the original research question, we pivot the dataframe so lockdown and pre are in separate columns

In [383]:
pivot_df= pd.pivot_table(final_df, values = 'activity level',
               index=['age group', 'country'], columns=["year"]).reset_index()

# add the percentage change from pre to lockdown
pivot_df["change"] = pivot_df["lockdown"]- pivot_df["pre"]

pivot_df

year,age group,country,lockdown,pre,change
0,12 to 17 years,Canada,42.9,55.5,-12.6
1,12 to 17 years,the Netherlands,42.4,41.7,0.7
2,18 to 34 years,Canada,59.9,64.3,-4.4
3,18 to 34 years,the Netherlands,62.2,59.9,2.3
4,35 to 49 years,Canada,57.2,58.4,-1.2
5,35 to 49 years,the Netherlands,60.5,55.6,4.9
6,50 to 64 years,Canada,55.1,54.0,1.1
7,50 to 64 years,the Netherlands,59.9,55.8,4.1
8,65 years and over,Canada,40.3,37.3,3.0
9,65 years and over,the Netherlands,56.0,55.1,0.9


In [384]:
#One way ANOVA
stats.f_oneway(pivot_df['change'][pivot_df['age group'] == '12 to 17 years'],
               pivot_df['change'][pivot_df['age group'] == '18 to 34 years'],
               pivot_df['change'][pivot_df['age group'] == '35 to 49 years'],
               pivot_df['change'][pivot_df['age group'] == '50 to 64 years'],
               pivot_df['change'][pivot_df['age group'] == '65 years and over'])

F_onewayResult(statistic=0.9254405286343624, pvalue=0.5164323432095254)

- There is no difference in changes of activity levels between age groups from pre to lockdown

Another interesting comparison is are the relative comparisons per age group. To compare this, we first add another column to the pivot_df

In [385]:
pivot_df["relative change"] = ((pivot_df["lockdown"]- pivot_df["pre"]) / pivot_df["lockdown"])*100
pivot_df

year,age group,country,lockdown,pre,change,relative change
0,12 to 17 years,Canada,42.9,55.5,-12.6,-29.370629
1,12 to 17 years,the Netherlands,42.4,41.7,0.7,1.650943
2,18 to 34 years,Canada,59.9,64.3,-4.4,-7.345576
3,18 to 34 years,the Netherlands,62.2,59.9,2.3,3.697749
4,35 to 49 years,Canada,57.2,58.4,-1.2,-2.097902
5,35 to 49 years,the Netherlands,60.5,55.6,4.9,8.099174
6,50 to 64 years,Canada,55.1,54.0,1.1,1.996370
7,50 to 64 years,the Netherlands,59.9,55.8,4.1,6.844741
8,65 years and over,Canada,40.3,37.3,3.0,7.444169
9,65 years and over,the Netherlands,56.0,55.1,0.9,1.607143


In [386]:
# One way ANOVA
stats.f_oneway(pivot_df['relative change'][pivot_df['age group'] == '12 to 17 years'],
               pivot_df['relative change'][pivot_df['age group'] == '18 to 34 years'],
               pivot_df['relative change'][pivot_df['age group'] == '35 to 49 years'],
               pivot_df['relative change'][pivot_df['age group'] == '50 to 64 years'],
               pivot_df['relative change'][pivot_df['age group'] == '65 years and over'])

F_onewayResult(statistic=0.9698419438846229, pvalue=0.4978308886147418)

- Again there are no significant differences between the relative changes in activity levels from pre to lockdown between age groups

# Visualization

In [390]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import Dark2_5 as palette
output_notebook()

def make_plot(final_df):
    # different color for each age group
    colors = palette
    count = 0
    # define the x_axis range
    years = ["pre", "lockdown"]
    p = figure(title="Comparing activity level changes across age groups before and during lockdown",
               background_fill_color="#fafafa", x_range=years)
    for age in final_df["age group"].unique(): # Iterate and create circle graph for each age group
        p.circle(y=final_df["activity level"][final_df["age group"]==age],
                 x=final_df["year"][final_df["age group"]==age],
                 legend_label=str(age), color=colors[count], alpha=0.8, 
                 radius=(final_df["standard error"][final_df["age group"]==age]/100)), # change the radius depending on SE
        count += 1
    
    p.xaxis.axis_label = 'Period'
    p.yaxis.axis_label = ">2.5 hours physical activities per week (%)"
    p.grid.grid_line_color="white"
    p.legend.location = "bottom_right"
    return p

plot = make_plot(final_df)
show(plot)

Loading BokehJS ...

# Conclusion

In conclusion, there were no significant differences between the continued physical activity during lockdown across age groups. A big caveat is that the variance was lost in the tests, which lowered the statistical power of these tests by a lot. I've tried to make it work, importing the standard error but I couldn't find a succesful way to incorporate them into the tests. My guess is, the 12-17 year group would have had a statistically signficant larger decrease in activity compared to the >40 age groups, with no differences between those groups at all.